In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import clear_output
import os
from selenium.webdriver.support import expected_conditions
import selenium
import itertools
import json
from sc_header import createDriver, agregar_links, scrape_url, scrape_batch, aplanar_lista,check_length, aplanar

In [2]:
start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

In [3]:
browser = createDriver()

In [4]:
browser.delete_all_cookies()

In [5]:
browser.get('https://www.shein.com/women-dresses-c-1727.html?icn=women-dresses&ici=www_tab01navbar05')
try:
    browser.execute_script('document.querySelector("body > div.welcome-privacy.j-welcome-privacy > div > div.c-modal-wrap.welocme-modal > div.c-modal > div > div > div.modal-body > div > div > button").click()')
except:
    pass
page = BeautifulSoup(browser.page_source,'html.parser')
url_base = 'https://www.shein.com'

In [6]:
#browser.delete_all_cookies()
links_sin_color =[['https://www.shein.com/women-dresses-c-1727.html?icn=women-dresses&ici=www_tab01navbar05',
                 'VESTIDO']]

In [7]:
Tlistas = []
for lsc in links_sin_color:
    
    agregar_links(lsc[0],
                 browser,
                 Tlistas,
                 lsc[1],
                 url_base)    

In [8]:
#browser.save_screenshot("asd.png")

In [9]:
hrefs_list = []
ex_col = []
for lista in Tlistas:
    try:
        browser.get(lista[2])
    except:
        browser.quit()
        time.sleep(1)
        browser = createDriver()
        browser.get(lista[2])
        
    
    while True:
        
        soup = BeautifulSoup(browser.page_source,'html.parser')
        indice_=0
        #for element in soup.find_all(class_='c-goodsitem__ratiowrap'):
        for element in soup.select('div.j-switch-color-wrap > div.c-goodsitem__ratiowrap > a'):
            href_aux = url_base +'/'+ element['href']

            hrefs_list.append([indice_,
                               lista[0],
                               lista[1],
                               lista[2],
                               href_aux
                              ])
            indice_+=1
                
                        
        try:
            browser.find_element_by_class_name('page-next').find_element_by_tag_name('a').click()
        except:
            break


In [10]:
df = pd.DataFrame(hrefs_list,columns=['pos','tipo','color','url_scraper','href'])

In [11]:
df['auxi'] = df['href'].apply(check_length)


In [12]:
len(df),len(df.drop_duplicates())

(8979, 8979)

In [13]:
if len(df['tipo'].unique().tolist()) != len(set([item[0] for item in Tlistas])):
    print('REVISAR LOS TIPOS SCRAPEADOS')
    raise

In [14]:
start_ = datetime.datetime.now()
items = []
errores = []

batch_size = 5
url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]

var = 0
for ix, url_chunk in enumerate(url_chunks):
    scrape_batch(errores,items,url_chunk)
    
    if int(ix/50) == ix/50:
        
        try:
            df = pd.DataFrame(items)
            df.to_excel('dress.xlsx')
            
            if len(errores) >0:
                
                pd.DataFrame(errores).to_excel('dress_error.xlsx')
                
            file = open("dress.txt", "w")
            file.write( str(url_chunk) + os.linesep)
            file.close()
        except:
            print("Erro al crear .xlsx")    
    
end_ = datetime.datetime.now()

In [15]:
# start_ = datetime.datetime.now()
# items = []

# batch_size = 5
# url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]

# var = 0
# for url_chunk in url_chunks:
#     items.append(scrape_batch(url_chunk))
#     var += 1
    
#     if var == 50:
#         var = 0
#         try:
#             df = pd.DataFrame(items)
#             df.to_excel('dress.xlsx')
                                                        
#             file = open("dress.txt", "w")
#             file.write( str(url_chunk) + os.linesep)
#             file.close()
#         except:
#             print("Erro al crear .xlsx")
    
# end_ = datetime.datetime.now()

In [16]:
print('Tiempo de ejecución: {}'.format(end_ - start_)[:-4])

Tiempo de ejecución: 11:44:37.18


In [17]:
new_list = aplanar_lista(items)
new_list = [i for i in new_list if type(i) != int]
dfShein = pd.DataFrame(new_list)

In [18]:
dfShein = dfShein.rename(columns={0: 'pos'})
dfShein = dfShein.rename(columns={1: 'Tipo'})
dfShein = dfShein.rename(columns={2: 'Color'})
dfShein = dfShein.rename(columns={3: 'id Producto'})
dfShein = dfShein.rename(columns={4: 'Descripcion'})
dfShein = dfShein.rename(columns={5: 'Talle'})
dfShein = dfShein.rename(columns={6: 'Stock'})
dfShein = dfShein.rename(columns={7: 'Precio'})
dfShein = dfShein.rename(columns={8: 'Precio_dto'})
dfShein = dfShein.rename(columns={9: 'Url Imagen'})
dfShein = dfShein.rename(columns={10: 'Url Producto'})
dfShein = dfShein.rename(columns={11: 'pagina_scraper'})

In [19]:
dfShein["Fecha"] = hoy
dfShein["Marca"] = "SHEIN"
dfShein["Moneda"] = "DOLAR US$"
dfShein['Sexo'] = 'Mujer'
dfShein['Descripcion aux'] = '0'

In [20]:

paginas = len(dfShein['pagina_scraper'].unique())
file = open("dress.txt", "a")
file.write(  os.linesep +f'CANTIDAD_ITEMS - {len(dfShein)},'+ f' DURACION - {format(end_ - start_)[:-4]},'+
           f' CANTIDAD_HOJAS - {paginas},'+ f' FECHA_ALTA - {hoy}' + os.linesep   
          )
file.close()

In [21]:
if len(dfShein['Tipo'].unique().tolist()) != len(set([item[0] for item in Tlistas])):
    print('REVISAR LOS TIPOS SCRAPEADOS')
    raise

In [22]:
dfShein.to_excel(f'../Salida/dfShein_dress{fecha}.xlsx')

In [23]:
len(dfShein),len(dfShein.dropna()),len(dfShein.drop_duplicates())

(36186, 36186, 36186)

In [24]:
# restore = pd.read_excel('dress.xlsx')
# lista_ = []
# for col in restore.columns:
#     for i in restore[col]:
#         lista_ += [[elementos.strip().replace('"', '').replace("'", '') for elementos in i.strip('[]').split('], [')[0].split(',')]]
        
# for index,row in df.iterrows():
#     if row['href'] == lista_[-1][-2]:
#         print(index+1)
#         break
# #df = df.loc[index+1:].copy()